<a href="https://colab.research.google.com/github/AgneseRe/Real-Time-Anomaly-Segmentation-for-Road-Scenes/blob/main/AML_AnomalySegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Real-time Anomaly Segmentation for Road Scenes**

Existing deep neural networks, when deployed in open-world settings, perform poorly on unknown, anomaly, out-of-distribution (OoD) objects that were not present during the training. The goal of this project is to build tiny anomaly segmentation models to segment anomaly patterns. Models must be able to fit in small devices, which represents a realistic memory constraint for an edge application.

## Preparation

In [1]:
!rm -r sample_data/

Install required packages and import useful modules.

In [2]:
%%capture
!pip3 install --quiet numpy
!pip3 install --quiet Pillow

!pip3 install --quiet gdown
!pip3 install --quiet torchvision
!pip3 install --quiet ood_metrics
!pip3 install --quiet cityscapesscripts

!pip3 install --quiet matplotlib
!pip3 install --quiet visdom

import os, sys, subprocess, torch

The following function is implemented to download the *Cityscapes* dataset in two different ways: via Google Drive (using `gdown`) or directly from the Cityscapes official website (using `csDownload`). Although the first option is preferable as it is definitely faster, direct download from the website is provided as an alternative. `gdown` may in fact raise the error *Failed to retrieve the file url* if the file we are attempting to download is exceptionally large (*e.g.* 11G), there are numerous users simultaneously trying to download it programmatically or we download it many times in a limited time. Regardless of the method used, use the conversor (available [here](https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/preparation/createTrainIdLabelImgs.py)) to generate labelTrainIds from labelIds.

In [3]:
def download_cityscapes():

    if not os.path.isdir('/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/cityscapes'):
        print("Attempting to download cityscapes dataset using gdown...")

        try:
            # If check is true, and the process exits with a non-zero exit code, a CalledProcessError exception will be raised.
            subprocess.run(["gdown", "https://drive.google.com/uc?id=11gSQ9UcLCnIqmY7srG2S6EVwV3paOMEq"], check=True)
            print("Dataset downloaded successfully using gdown. Unzipping...")
            subprocess.run(["unzip", "-q", "cityscapes.zip"], check=True)
            # Use the conversor to generate labelTrainIds from labelIds
            print("Generating trainIds from labelIds...")
            !CITYSCAPES_DATASET='cityscapes/' csCreateTrainIdLabelImgs

        except subprocess.CalledProcessError as e:
            print("gdown failed. Attempting to download cityscapes dataset from the official website...")
            try:
              !csDownload leftImg8bit_trainvaltest.zip
              !csDownload gtFine_trainvaltest.zip

              print("Dataset downloaded successfully from the official website. Unzipping...")
              !unzip -q 'leftImg8bit_trainvaltest.zip' -d 'cityscapes'
              !unzip -o -q 'gtFine_trainvaltest.zip' -d 'cityscapes'

              print("Generating trainIds from labelIds...")
              !CITYSCAPES_DATASET='cityscapes/' csCreateTrainIdLabelImgs

              print("Cityscapes dataset ready")

            except Exception as e2:
                print("Failed to download the dataset using both methods.")

Download and unzip the validation dataset (*FS_LostFound_full*, *RoadAnomaly*, *RoadAnomaly21*, *RoadObsticle21*, *fs_static*), clone or update the GitHub repository (*Real-Time-Anomaly-Segmentation-for-Road-Scenes*) and download the *Cityscapes* dataset.

In [4]:
# download and unzip validation dataset
if not os.path.isdir('/content/validation_dataset'):
  !gdown 'https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta'
  !unzip -q 'Validation_Dataset.zip'
  !mkdir validation_dataset && cp -pR Validation_Dataset/* validation_dataset/ && rm -R Validation_Dataset/
  !rm 'Validation_Dataset.zip'

# clone the github repo and pull command
if not os.path.isdir('content/Real-Time-Anomaly-Segmentation-for-Road-Scenes'):
  !git clone https://github.com/AgneseRe/Real-Time-Anomaly-Segmentation-for-Road-Scenes.git
else: # if folder already present
  !git pull

%cd Real-Time-Anomaly-Segmentation-for-Road-Scenes

Downloading...
From (original): https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta
From (redirected): https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta&confirm=t&uuid=c042e90b-0d5a-4a28-a862-3894735f0c3e
To: /content/Validation_Dataset.zip
100% 329M/329M [00:03<00:00, 94.0MB/s]
Cloning into 'Real-Time-Anomaly-Segmentation-for-Road-Scenes'...
remote: Enumerating objects: 1922, done.
remote: Total 1922 (delta 0), reused 0 (delta 0), pack-reused 1922 (from 1)
Receiving objects: 100% (1922/1922), 2.15 GiB | 39.13 MiB/s, done.
Resolving deltas: 100% (1040/1040), done.
Updating files: 100% (471/471), done.
/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes


In [5]:
# download cityscapes dataset - use credentials (agnesere, FCSBwcVMi-u9-Zn) if downloading from official site
download_cityscapes()

Attempting to download cityscapes dataset using gdown...
gdown failed. Attempting to download cityscapes dataset from the official website...
Cityscapes username or email address: agnesere
Cityscapes password: 
Store credentials unencrypted in '/root/.local/share/cityscapesscripts/credentials.json' [y/N]: N
Download progress:  98% 10.8G/11.0G [08:51<00:10, 21.8MB/s]
Cityscapes username or email address: agnesere
Cityscapes password: 
Store credentials unencrypted in '/root/.local/share/cityscapesscripts/credentials.json' [y/N]: N
Download progress: 100% 241M/241M [00:12<00:00, 19.8MB/s]
Dataset downloaded successfully from the official website. Unzipping...
Generating trainIds from labelIds...
Processing 5000 annotation files
Progress: 100.0 % Cityscapes dataset ready


## Evaluation

### Step 2A

#### Compute AuPRC & FPR95TPR

In [6]:
%cd eval

/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval


Define datasets used for evaluation.

In [7]:
# datasets = os.listdir("../../validation_dataset")
datasets = {
    "SMIYC RA-21": "RoadAnomaly21",
    "SMIYC RO-21": "RoadObsticle21",
    "FS L&F": "FS_LostFound_full",
    " FS Static": "fs_static",
    "Road Anomaly": "RoadAnomaly"
    }

List anomaly detection methods used for evaluation.

In [8]:
methods = ["MSP", "MaxLogit", "MaxEntropy"]

Automate running anomaly detection experiments on multiple datasets using different methods. The evaluation script can be invoked with the appropriate parameters: name of the model, path to the training folder, base folder to save generated plots, directory to load the model.

In [9]:
def run_eval_anomaly(datasets, methods, model = None, training_folder = None, plot_folder = None, load_dir = None) -> None:

  for dataset, folder in datasets.items():
    print(f"Dataset {dataset}")

    for method in methods:
      print(f" - {method:<10} ", end = "")
      input_path = f"../../validation_dataset/{folder}/images/*.*"
      plot_dir_path = f"../plots/losses/{plot_folder}/{folder}_{method}" if model else f"../plots/baselines/{folder}_{method}"

      add_cmd = "--cpu" if not torch.cuda.is_available() else ""

      if model:
        !python evalAnomaly.py --input={input_path} --method={method} --loadModel={model} --loadDir={load_dir} --loadWeights={training_folder}/model_best.pth --plotdir={plot_dir_path} {add_cmd}
      else: # ERFNet pre-trained
        !python evalAnomaly.py --input={input_path} --method={method} --plotdir={plot_dir_path} {add_cmd}

    print("=" * 55, end = "\n")

Evaluate a segmentation model on Cityscapes using specified weights.

In [10]:
def run_eval_iou(model = "erfnet", load_dir = "../trained_models/", training_folder = "erfnet_pretrained.pth", void = False) -> None:

  load_model = f"{model}.py" if model != "erfnet_isomaxplus" else "erfnet.py"
  add_cmd = "--cpu" if not torch.cuda.is_available() else ""
  method_flag = "--method void" if void else "" # for void classifier
  !python eval_iou.py --model={model} --loadDir={load_dir} --loadModel={load_model} --loadWeights={training_folder} --datadir /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/cityscapes {method_flag} {add_cmd}

Perform inference using the pre-trained **ERFNet** model on anomaly segmentation test datasets provided. Evaluate results with different techniques: MSP, MaxLogit and MaxEntropy.

In [ ]:
run_eval_anomaly(datasets, methods)

Dataset SMIYC RA-21
 - MSP        | AUPRC score: 29.100 | FPR@TPR95: 62.511
 - MaxLogit   | AUPRC score: 38.320 | FPR@TPR95: 59.337
 - MaxEntropy | AUPRC score: 31.005 | FPR@TPR95: 62.593
Dataset SMIYC RO-21
 - MSP        | AUPRC score: 2.712 | FPR@TPR95: 64.974
 - MaxLogit   | AUPRC score: 4.627 | FPR@TPR95: 48.443
 - MaxEntropy | AUPRC score: 3.052 | FPR@TPR95: 65.600
Dataset FS L&F
 - MSP        | AUPRC score: 1.748 | FPR@TPR95: 50.763
 - MaxLogit   | AUPRC score: 3.301 | FPR@TPR95: 45.495
 - MaxEntropy | AUPRC score: 2.582 | FPR@TPR95: 50.368
Dataset  FS Static
 - MSP        | AUPRC score: 7.470 | FPR@TPR95: 41.823
 - MaxLogit   | AUPRC score: 9.499 | FPR@TPR95: 40.300
 - MaxEntropy | AUPRC score: 8.826 | FPR@TPR95: 41.523
Dataset Road Anomaly
 - MSP        | AUPRC score: 12.426 | FPR@TPR95: 82.492
 - MaxLogit   | AUPRC score: 15.582 | FPR@TPR95: 73.248
 - MaxEntropy | AUPRC score: 12.678 | FPR@TPR95: 82.632


If you want to save the baselines folder in your local machine, create a ZIP file with the following command and then download it.

In [ ]:
 # !zip -r baselines.zip baselines/

#### Compute mIoU

In [ ]:
run_eval_iou()

Loading model: ../trained_models/erfnet
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
---------------------------------------
Took  80.3807921409607 seconds
Per-Class IoU:
97.62 Road
81.37 sidewalk
90.77 building
49.43 wall
54.93 fence
60.81 pole
62.60 traffic light
72.32 traffic sign
91.35 vegetation
60.97 terrain
93.38 sky
76.11 person
53.45 rider
92.91 car
72.78 truck
78.87 bus
63.86 train
46.41 motorcycle
71.89 bicycle
MEAN IoU:  72.20 %


### Step 2B

#### Compute AuPRC & FPR95TPR with temperature scaling

In [ ]:
temperatures = [0.5, 0.75, 1.0, 1.1, 1.2, 1.5, 2.0, 5.0, 10.0]

for dataset, folder in datasets.items():
  print(f"Dataset {dataset}")

  for temperature in temperatures:
    print(f" - {temperature:<10} ", end = "")
    input_path = f"../../validation_dataset/{folder}/images/*.*"
    if torch.cuda.is_available():
      !python evalAnomaly.py --input={input_path} --method="MSP" --temperature={temperature}
    else:
      !python evalAnomaly.py --input={input_path} --method="MSP" --temperature={temperature} --cpu

  print("=" * 55, end = "\n")

Dataset SMIYC RA-21
 - 0.5        | AUPRC score: 27.061 | FPR@TPR95: 62.731
 - 0.75       | AUPRC score: 28.156 | FPR@TPR95: 62.479
 - 1.0        | AUPRC score: 29.100 | FPR@TPR95: 62.511
 - 1.1        | AUPRC score: 29.410 | FPR@TPR95: 62.590
 - 1.2        | AUPRC score: 29.678 | FPR@TPR95: 62.724
 - 1.5        | AUPRC score: 30.258 | FPR@TPR95: 63.318
 - 2.0        | AUPRC score: 30.679 | FPR@TPR95: 64.721
 - 5.0        | AUPRC score: 30.196 | FPR@TPR95: 71.594
 - 10.0       | AUPRC score: 29.526 | FPR@TPR95: 75.757
Dataset SMIYC RO-21
 - 0.5        | AUPRC score: 2.420 | FPR@TPR95: 63.225
 - 0.75       | AUPRC score: 2.567 | FPR@TPR95: 64.053
 - 1.0        | AUPRC score: 2.712 | FPR@TPR95: 64.974
 - 1.1        | AUPRC score: 2.766 | FPR@TPR95: 65.524
 - 1.2        | AUPRC score: 2.816 | FPR@TPR95: 66.033
 - 1.5        | AUPRC score: 2.937 | FPR@TPR95: 67.928
 - 2.0        | AUPRC score: 3.026 | FPR@TPR95: 71.459
 - 5.0        | AUPRC score: 2.841 | FPR@TPR95: 83.111
 - 10.0       | 

### Training models

We explored two training strategies:
  - **Training from scratch** on the Cityscapes dataset.
  - **Fine-tuning** a pretrained model. Leverage pretrained versions of the models, and further fine-tune them to adapt to our task.

### Utils

In [11]:
base_dir = "../train"
data_dir = "../cityscapes"

In [12]:
def train_model(model: str, num_epochs: int, batch_size: int, stop_epoch: int = 20, pretrained: bool = False, resume: bool = False, fineTune: bool = False) -> None:

  state_flag = f"--state ../trained_models/{model}_pretrained.pth" if pretrained else ""
  resume_flag = "--resume" if resume else ""
  finetune_flag = f"--FineTune --loadWeights ../trained_models/{model}_pretrained.pth" if fineTune else ""

  # if model == "bisenet":
  #     !gdown "https://drive.usercontent.google.com/download?id=1Gj4eZrmdygA5c_y7N0KrmSRThoYjfjk-" -O "checkpoint.pth.tar"

  if fineTune:
    savedir_name = f"{model}_training_void_ft"
  else:
    savedir_name = f"{model}_training_void"

  !cd {base_dir} && python -W ignore main_v2.py \
    --savedir {savedir_name}\
    --datadir {data_dir} \
    --model {model} \
    --cuda \
    --num-epochs={num_epochs} \
    --epochs-save=1 \
    --batch-size={batch_size} \
    --stop-epoch={stop_epoch} \
    --decoder \
    {finetune_flag} \
    {state_flag} \
    {resume_flag}

### ERFNet Training

Resumed ERFNet training from a saved checkpoint due to Colab's session time limitations. The encoder and the first few decoder epochs were trained in a previous session. The training of the remaining part of the decoder is detailed below.

The encoder was trained for 20 epochs, followed by the decoder, also trained for 20 epochs.

In [ ]:
train_model("erfnet", num_epochs=20, batch_size=6)

In [13]:
train_model("erfnet", num_epochs=20, batch_size=6, resume=True)
# %cd ../save
# !zip -r erfnet_training_void.zip erfnet_training_void/

========== TRAINING ===========
========== DECODER TRAINING ===========
../cityscapes/leftImg8bit/train
../cityscapes/leftImg8bit/val
Criterion: CrossEntropyLoss2d
=> Loaded checkpoint at epoch 7)
----- TRAINING - EPOCH 7 -----
LEARNING RATE:  0.00036270892346860996
loss: 0.7219 (epoch: 7, step: 0) // Avg time/img: 0.4481 s
loss: 0.4191 (epoch: 7, step: 50) // Avg time/img: 0.1129 s
loss: 0.4132 (epoch: 7, step: 100) // Avg time/img: 0.1105 s
loss: 0.4109 (epoch: 7, step: 150) // Avg time/img: 0.1108 s
loss: 0.4086 (epoch: 7, step: 200) // Avg time/img: 0.1111 s
loss: 0.4134 (epoch: 7, step: 250) // Avg time/img: 0.1118 s
loss: 0.4095 (epoch: 7, step: 300) // Avg time/img: 0.1124 s
loss: 0.4031 (epoch: 7, step: 350) // Avg time/img: 0.1128 s
loss: 0.4017 (epoch: 7, step: 400) // Avg time/img: 0.1131 s
loss: 0.4001 (epoch: 7, step: 450) // Avg time/img: 0.1134 s
----- VALIDATING - EPOCH 7 -----
VAL loss: 0.4987 (epoch: 7, step: 0) // Avg time/img: 0.0358 s
VAL loss: 0.5223 (epoch: 7, st

### ERFNet Fine-Tuning

Initiate the fine-tuning process of ERFNet over 20 epochs with a batch size of 6, setting the parameter `fineTune=True`. The pretrained ERFNet model on the Cityscapes dataset is fine-tuned including the 20th void class, which represents background or unannotated regions.

In [ ]:
train_model("erfnet", num_epochs=20, batch_size=6, fineTune=True)
# %cd ../save
# !zip -r erfnet_training_void_ft.zip erfnet_training_void_ft/

Import Model erfnet with weights ../trained_models/erfnet_pretrained.pth to FineTune
========== TRAINING ===========
../cityscapes/leftImg8bit/train
../cityscapes/leftImg8bit/val
Criterion: <class 'utils.losses.ce_loss.CrossEntropyLoss2d'>
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  5e-05
loss: 0.5082 (epoch: 1, step: 0) // Avg time/img: 0.3150 s
loss: 0.3931 (epoch: 1, step: 50) // Avg time/img: 0.0396 s
loss: 0.3815 (epoch: 1, step: 100) // Avg time/img: 0.0371 s
loss: 0.3817 (epoch: 1, step: 150) // Avg time/img: 0.0363 s
loss: 0.3752 (epoch: 1, step: 200) // Avg time/img: 0.0360 s
loss: 0.3759 (epoch: 1, step: 250) // Avg time/img: 0.0357 s
loss: 0.3744 (epoch: 1, step: 300) // Avg time/img: 0.0355 s
loss: 0.3768 (epoch: 1, step: 350) // Avg time/img: 0.0353 s
loss: 0.3742 (epoch: 1, step: 400) // Avg time/img: 0.0353 s
loss: 0.3715 (epoch: 1, step: 450) // Avg time/img: 0.0352 s
----- VALIDATING - EPOCH 1 -----
VAL loss: 0.4713 (epoch: 1, step: 0) // Avg time/img: 0.0335 s
VAL 

### BiSeNet Training

Instead of directly fine-tuning a pretrained model, we opted to train BiSeNet from scratch for 40 epochs.

In the first run, due to GPU time limitations on Google Colab, the training is intentionally interrupted after 20 epochs by setting the parameter `stop_epoch` equal to 20. Remember to set `num_epochs` to 40 from the beginning to ensure that the learning rate scheduler behaves correctly across the full training process. The process is then resumed in the following run from epoch 21 using the `--resume` flag.

In [ ]:
train_model("bisenet", num_epochs=40, batch_size=6, stop_epoch=20)
# %cd ../save
# !zip -r bisenet_training_void.zip bisenet_training_void/

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100% 44.7M/44.7M [00:00<00:00, 301MB/s]
========== TRAINING ===========
../cityscapes/leftImg8bit/train
../cityscapes/leftImg8bit/val
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.025
loss: 5.835 (epoch: 1, step: 0) // Avg time/img: 0.4772 s
loss: 3.602 (epoch: 1, step: 50) // Avg time/img: 0.0502 s
loss: 3.355 (epoch: 1, step: 100) // Avg time/img: 0.0460 s
loss: 3.206 (epoch: 1, step: 150) // Avg time/img: 0.0446 s
loss: 3.098 (epoch: 1, step: 200) // Avg time/img: 0.0440 s
loss: 3.043 (epoch: 1, step: 250) // Avg time/img: 0.0438 s
loss: 2.982 (epoch: 1, step: 300) // Avg time/img: 0.0437 s
loss: 2.926 (epoch: 1, step: 350) // Avg time/img: 0.0436 s
loss: 2.897 (epoch: 1, step: 400) // Avg time/img: 0.0435 s
loss: 2.871 (epoch: 1, step: 450) // Avg time/img: 0.0434 s
----- VALIDATING - EPOCH 1 -----
VAL loss: 2.136 (epoch: 1, step: 0) // Avg time/im

In [ ]:
train_model("bisenet", num_epochs=40, batch_size=6, stop_epoch=40, resume=True)
# %cd ../save
# !zip -r bisenet_training_void.zip bisenet_training_void/

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100% 44.7M/44.7M [00:00<00:00, 384MB/s]
========== TRAINING ===========
../cityscapes/leftImg8bit/train
../cityscapes/leftImg8bit/val
=> Loaded checkpoint at epoch 21)
----- TRAINING - EPOCH 21 -----
LEARNING RATE:  0.013397168281703665
loss: 2.068 (epoch: 21, step: 0) // Avg time/img: 0.4451 s
loss: 2.012 (epoch: 21, step: 50) // Avg time/img: 0.0482 s
loss: 2.009 (epoch: 21, step: 100) // Avg time/img: 0.0447 s
loss: 2.011 (epoch: 21, step: 150) // Avg time/img: 0.0438 s
loss: 2.021 (epoch: 21, step: 200) // Avg time/img: 0.0436 s
loss: 2.012 (epoch: 21, step: 250) // Avg time/img: 0.0436 s
loss: 2.017 (epoch: 21, step: 300) // Avg time/img: 0.0436 s
loss: 2.026 (epoch: 21, step: 350) // Avg time/img: 0.0435 s
loss: 2.02 (epoch: 21, step: 400) // Avg time/img: 0.0436 s
loss: 2.019 (epoch: 21, step: 450) // Avg time/img: 0.0435 s
----- VALIDATING - EPOCH

### BiSeNet Fine-Tuning

Fine-tune a pretrained BiSeNet model on the Cityscapes dataset for 20 epochs by setting `fineTune=True`.

In [ ]:
train_model("bisenet", num_epochs=20, batch_size=6, fineTune=True)
# %cd ../save
# !zip -r bisenet_training_void_ft.zip bisenet_training_void_ft/

Import Model bisenet with weights ../trained_models/bisenet_pretrained.pth to FineTune
========== TRAINING ===========
../cityscapes/leftImg8bit/train
../cityscapes/leftImg8bit/val
Criterion: <class 'utils.losses.ohem_ce_loss.OhemCELoss'>
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.0025
loss: 6.09 (epoch: 1, step: 0) // Avg time/img: 0.4535 s
loss: 5.907 (epoch: 1, step: 50) // Avg time/img: 0.0267 s
loss: 5.636 (epoch: 1, step: 100) // Avg time/img: 0.0225 s
loss: 5.524 (epoch: 1, step: 150) // Avg time/img: 0.0203 s
loss: 5.437 (epoch: 1, step: 200) // Avg time/img: 0.0195 s
loss: 5.399 (epoch: 1, step: 250) // Avg time/img: 0.0191 s
loss: 5.411 (epoch: 1, step: 300) // Avg time/img: 0.0187 s
loss: 5.409 (epoch: 1, step: 350) // Avg time/img: 0.0183 s
loss: 5.387 (epoch: 1, step: 400) // Avg time/img: 0.0182 s
loss: 5.391 (epoch: 1, step: 450) // Avg time/img: 0.0180 s
----- VALIDATING - EPOCH 1 -----
VAL loss: 7.204 (epoch: 1, step: 0) // Avg time/img: 0.0365 s
VAL loss: 6.475 

### ENet Training

Same procedure adopted for BiSeNet is applied for ENet here.

In [ ]:
train_model("enet", num_epochs=40, batch_size=6, stop_epoch=20)
# %cd ../save
# !zip -r enet_training_void.zip enet_training_void/

========== TRAINING ===========
../cityscapes/leftImg8bit/train
../cityscapes/leftImg8bit/val
Criterion: CrossEntropyLoss2d
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.0005
loss: 3.071 (epoch: 1, step: 0) // Avg time/img: 0.4825 s
loss: 2.726 (epoch: 1, step: 50) // Avg time/img: 0.0765 s
loss: 2.334 (epoch: 1, step: 100) // Avg time/img: 0.0721 s
loss: 2.05 (epoch: 1, step: 150) // Avg time/img: 0.0711 s
loss: 1.858 (epoch: 1, step: 200) // Avg time/img: 0.0707 s
loss: 1.716 (epoch: 1, step: 250) // Avg time/img: 0.0707 s
loss: 1.613 (epoch: 1, step: 300) // Avg time/img: 0.0705 s
loss: 1.531 (epoch: 1, step: 350) // Avg time/img: 0.0707 s
loss: 1.462 (epoch: 1, step: 400) // Avg time/img: 0.0707 s
loss: 1.406 (epoch: 1, step: 450) // Avg time/img: 0.0707 s
----- VALIDATING - EPOCH 1 -----
VAL loss: 0.7688 (epoch: 1, step: 0) // Avg time/img: 0.0339 s
VAL loss: 0.9467 (epoch: 1, step: 50) // Avg time/img: 0.0316 s
EPOCH IoU on VAL set:  17.06 %
Saving model as best
save: ../save/

In [ ]:
train_model("enet", num_epochs=40, batch_size=6, stop_epoch=40, resume=True)
# %cd ../save
# !zip -r enet_training_void.zip enet_training_void/

========== TRAINING ===========
../cityscapes/leftImg8bit/train
../cityscapes/leftImg8bit/val
Criterion: CrossEntropyLoss2d
=> Loaded checkpoint at epoch 21)
----- TRAINING - EPOCH 21 -----
LEARNING RATE:  0.0005
loss: 0.3133 (epoch: 21, step: 0) // Avg time/img: 0.4467 s
loss: 0.423 (epoch: 21, step: 50) // Avg time/img: 0.0767 s
loss: 0.4169 (epoch: 21, step: 100) // Avg time/img: 0.0734 s
loss: 0.4144 (epoch: 21, step: 150) // Avg time/img: 0.0724 s
loss: 0.42 (epoch: 21, step: 200) // Avg time/img: 0.0723 s
loss: 0.4207 (epoch: 21, step: 250) // Avg time/img: 0.0722 s
loss: 0.4208 (epoch: 21, step: 300) // Avg time/img: 0.0719 s
loss: 0.4202 (epoch: 21, step: 350) // Avg time/img: 0.0720 s
loss: 0.4216 (epoch: 21, step: 400) // Avg time/img: 0.0720 s
loss: 0.4243 (epoch: 21, step: 450) // Avg time/img: 0.0721 s
----- VALIDATING - EPOCH 21 -----
VAL loss: 0.3167 (epoch: 21, step: 0) // Avg time/img: 0.0396 s
VAL loss: 0.4666 (epoch: 21, step: 50) // Avg time/img: 0.0312 s
EPOCH IoU 

###Enet Fine-Tuning

In [ ]:
train_model("enet", num_epochs=20, batch_size=6, fineTune=True)
# %cd ../save
# !zip -r enet_training_void_ft.zip enet_training_void_ft/

Import Model enet with weights ../trained_models/enet_pretrained.pth to FineTune
========== TRAINING ===========
../cityscapes/leftImg8bit/train
../cityscapes/leftImg8bit/val
Criterion: CrossEntropyLoss2d
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  5e-05
loss: 10.75 (epoch: 1, step: 0) // Avg time/img: 0.3722 s
loss: 10.32 (epoch: 1, step: 50) // Avg time/img: 0.0422 s
loss: 10.23 (epoch: 1, step: 100) // Avg time/img: 0.0380 s
loss: 10.13 (epoch: 1, step: 150) // Avg time/img: 0.0369 s
loss: 10.05 (epoch: 1, step: 200) // Avg time/img: 0.0360 s
loss: 9.928 (epoch: 1, step: 250) // Avg time/img: 0.0355 s
loss: 9.824 (epoch: 1, step: 300) // Avg time/img: 0.0357 s
loss: 9.703 (epoch: 1, step: 350) // Avg time/img: 0.0355 s
loss: 9.593 (epoch: 1, step: 400) // Avg time/img: 0.0355 s
loss: 9.481 (epoch: 1, step: 450) // Avg time/img: 0.0353 s
----- VALIDATING - EPOCH 1 -----
VAL loss: 8.499 (epoch: 1, step: 0) // Avg time/img: 0.0389 s
VAL loss: 8.385 (epoch: 1, step: 50) // Avg time/i

## Void Classifier

In [16]:
%cd ../eval

/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval


In [17]:
models = ["erfnet", "bisenet", "enet"]

### Evaluation: `eval_anomaly`

Run the `eval_anomaly` script on the models trained from scratch. All trained on the Cityscapes dataset including the 20th *void* class.

In [19]:
for model in models:
  print(f"{'=' * 64}")
  print(f"MODEL: {model.upper()}")
  print(f"{'=' * 64}")

  for dataset, folder in datasets.items():
    print(f"Dataset {dataset:<15}", end = "")

    input_path = f"../../validation_dataset/{folder}/images/*.*"
    plot_dir_path = f"../plots/void/{model}/{folder}"
    load_dir = f"../save/{model}_training_void/"
    load_weights = f"model_best.pth"

    add_cmd = "--cpu" if not torch.cuda.is_available() else ""
    !python evalAnomaly.py --input={input_path} --loadModel={model} --loadDir={load_dir} --loadWeights={load_weights} --method="void" --plotdir={plot_dir_path} {add_cmd}

  print()

MODEL: ERFNET
Dataset SMIYC RA-21    | AUPRC score: 10.950 | FPR@TPR95: 91.396
Dataset SMIYC RO-21    | AUPRC score:  0.917 | FPR@TPR95: 87.629
Dataset FS L&F         | AUPRC score:  5.658 | FPR@TPR95: 51.060
Dataset  FS Static     | AUPRC score:  7.318 | FPR@TPR95: 47.542
Dataset Road Anomaly   | AUPRC score:  6.999 | FPR@TPR95: 94.774

MODEL: BISENET
Dataset SMIYC RA-21    | AUPRC score: 25.768 | FPR@TPR95: 85.145
Dataset SMIYC RO-21    | AUPRC score: 10.697 | FPR@TPR95: 89.987
Dataset FS L&F         | AUPRC score: 15.784 | FPR@TPR95: 60.907
Dataset  FS Static     | AUPRC score:  6.232 | FPR@TPR95: 86.029
Dataset Road Anomaly   | AUPRC score: 12.276 | FPR@TPR95: 94.143

MODEL: ENET
Dataset SMIYC RA-21    | AUPRC score: 12.348 | FPR@TPR95: 84.774
Dataset SMIYC RO-21    | AUPRC score:  1.139 | FPR@TPR95: 63.600
Dataset FS L&F         | AUPRC score:  2.102 | FPR@TPR95: 65.472
Dataset  FS Static     | AUPRC score:  7.246 | FPR@TPR95: 39.319
Dataset Road Anomaly   | AUPRC score: 10.377 | 

Run the `eval_anomaly` script on the fine-tuned versions of ERFNet, BiSeNet, and ENet. Compare the performance with the previous ones.

In [ ]:
for model in models:
  print(f"{'=' * 64}")
  print(f"MODEL: {model.upper()}")
  print(f"{'=' * 64}")

  for dataset, folder in datasets.items():
    print(f"Dataset {dataset:<15}", end = "")

    input_path = f"../../validation_dataset/{folder}/images/*.*"
    plot_dir_path = f"../plots/void/{model}/{folder}"
    load_dir = f"../save/{model}_training_void_ft/"
    load_weights = f"model_best.pth"

    add_cmd = "--cpu" if not torch.cuda.is_available() else ""
    !python evalAnomaly.py --input={input_path} --loadModel={model} --loadDir={load_dir} --loadWeights={load_weights} --method="void" --plotdir={plot_dir_path} {add_cmd}

  print()

MODEL: ERFNET
Dataset SMIYC RA-21    | AUPRC score: 24.239 | FPR@TPR95: 68.735
Dataset SMIYC RO-21    | AUPRC score:  1.360 | FPR@TPR95: 99.809
Dataset FS L&F         | AUPRC score: 11.535 | FPR@TPR95: 15.749
Dataset  FS Static     | AUPRC score: 15.785 | FPR@TPR95: 49.445
Dataset Road Anomaly   | AUPRC score: 10.978 | FPR@TPR95: 86.823

MODEL: BISENET
Dataset SMIYC RA-21    | AUPRC score: 33.084 | FPR@TPR95: 86.463
Dataset SMIYC RO-21    | AUPRC score: 13.214 | FPR@TPR95: 99.474
Dataset FS L&F         | AUPRC score: 15.462 | FPR@TPR95: 52.662
Dataset  FS Static     | AUPRC score: 30.864 | FPR@TPR95: 60.263
Dataset Road Anomaly   | AUPRC score: 13.042 | FPR@TPR95: 93.016

MODEL: ENET
Dataset SMIYC RA-21    | AUPRC score: 21.616 | FPR@TPR95: 88.547
Dataset SMIYC RO-21    | AUPRC score:  1.763 | FPR@TPR95: 95.070
Dataset FS L&F         | AUPRC score:  0.624 | FPR@TPR95: 68.780
Dataset  FS Static     | AUPRC score:  6.430 | FPR@TPR95: 68.681
Dataset Road Anomaly   | AUPRC score: 18.122 | 

If you want to save the void folder in your local machine, create a ZIP file with the following command and then download it.

In [ ]:
# %cd ../plots
# !zip -r void.zip void/

### Evaluation: `eval_iou`

Run the `eval_iou` script on the models trained from scratch. Set the parameter `void=True`.

In [20]:
for model in models:
  print(f"{'=' * 64}")
  print(f"MODEL: {model.upper()}")
  print(f"{'=' * 64}")
  load_dir = f"../save/{model}_training_void/"
  training_folder = f"model_best.pth"

  run_eval_iou(model, load_dir, training_folder, void=True)
  print()

MODEL: ERFNET
Loading model: ../save/erfnet_training_void/erfnet.py
Loading weights: ../save/erfnet_training_void/model_best.pth
Model and weights LOADED successfully
---------------------------------------
Took  76.34682869911194 seconds
Per-Class IoU:
91.38 Road
65.77 sidewalk
83.28 building
36.07 wall
38.81 fence
45.63 pole
40.41 traffic light
50.80 traffic sign
87.45 vegetation
51.84 terrain
86.88 sky
58.70 person
35.58 rider
85.85 car
40.57 truck
50.13 bus
28.49 train
25.68 motorcycle
54.59 bicycle
60.83 void
MEAN IoU:  55.94 %

MODEL: BISENET
Loading model: ../save/bisenet_training_void/bisenet.py
Loading weights: ../save/bisenet_training_void/model_best.pth
Model and weights LOADED successfully
---------------------------------------
Took  73.1594660282135 seconds
Per-Class IoU:
92.31 Road
66.50 sidewalk
83.43 building
34.76 wall
35.86 fence
37.21 pole
42.08 traffic light
55.06 traffic sign
86.65 vegetation
51.33 terrain
87.12 sky
64.02 person
41.01 rider
88.21 car
51.31 truck
6

Run the `eval_iou` script on the fine-tuned versions of ERFNet, BiSeNet, and ENet. Compare the performance with the previous ones.



In [ ]:
for model in models:
  print(f"{'=' * 64}")
  print(f"MODEL: {model.upper()}")
  print(f"{'=' * 64}")
  load_dir = f"../save/{model}_training_void_ft/"
  training_folder = f"model_best.pth"

  run_eval_iou(model, load_dir, training_folder, void=True)
  print()

MODEL: ERFNET
Loading model: ../save/erfnet_training_void_ft/erfnet.py
Loading weights: ../save/erfnet_training_void_ft/model_best.pth
Model and weights LOADED successfully
---------------------------------------
Took  73.02247381210327 seconds
Per-Class IoU:
83.17 Road
66.71 sidewalk
83.86 building
35.33 wall
43.69 fence
55.00 pole
56.69 traffic light
62.44 traffic sign
88.13 vegetation
45.99 terrain
86.45 sky
68.59 person
52.79 rider
87.17 car
66.92 truck
74.83 bus
50.83 train
37.98 motorcycle
63.46 bicycle
14.08 void
MEAN IoU:  61.21 %

MODEL: BISENET
Loading model: ../save/bisenet_training_void_ft/bisenet.py
Loading weights: ../save/bisenet_training_void_ft/model_best.pth
Model and weights LOADED successfully
---------------------------------------
Took  70.93619394302368 seconds
Per-Class IoU:
86.76 Road
63.12 sidewalk
83.16 building
42.88 wall
37.57 fence
44.78 pole
49.69 traffic light
58.74 traffic sign
86.15 vegetation
50.23 terrain
86.78 sky
66.56 person
48.65 rider
89.20 car


✅ The scratch-trained model shows a much better IoU on the void class (61.23 and 64.75 against 38.62 and 7.35), suggesting that starting from scratch may lead to stronger learning of unannotated/anomalous regions—possibly due to no bias from pretrained features.

## Effect of Training Loss function

Analyze the effect of the training model along with losses that are specifically made for anomaly detection.

### Utils

In [ ]:
def train_erfnet_with_loss(base_dir: str, data_dir: str, loss: str, stop_epoch: int, num_epochs: int = 20,
                           batch_size: int = 6, resume: bool = False, logit_norm: bool = False,
                           iso_max: bool = False, class_weights: str = 'hard') -> None:

    resume_flag = "--resume" if resume else ""
    model = "erfnet_isomaxplus" if iso_max else "erfnet"
    logit_suffix = "_logit_norm" if logit_norm else ""
    logit_norm_flag = "--logit_norm" if logit_norm else ""
    pretrained_encoder = "../trained_models/erfnet_encoder_pretrained.pth.tar"

    !cd {base_dir} && python -W ignore main_v2.py \
      --savedir {model}_training_{loss}{logit_suffix}_final \
      --loss {loss} \
      --datadir {data_dir} \
      --model {model} \
      --cuda \
      --num-epochs={num_epochs} \
      --epochs-save=1 \
      --stop-epoch={stop_epoch} \
      --batch-size={batch_size} \
      {resume_flag} \
      {logit_norm_flag} \
      --decoder \
      --pretrainedEncoder={pretrained_encoder}

Different combinations of loss functions are experimented here.

### Cross-Entropy

In [ ]:
train_erfnet_with_loss(base_dir, data_dir, loss="ce")
# !zip -r erfnet_training_ce.zip erfnet_training_ce/

###Focal Loss


In [ ]:
train_erfnet_with_loss(base_dir, data_dir, loss="f")
# !zip -r erfnet_training_f.zip erfnet_training_f/

### Cross-Entropy + Focal Loss

In [ ]:
train_erfnet_with_loss(base_dir, data_dir, loss="cef")
# !zip -r erfnet_training_ce_focal.zip erfnet_training_ce_focal/

### Cross-Entropy + Logit Norm

In [ ]:
train_erfnet_with_loss(base_dir, data_dir, loss="ce", logit_norm=True)
# !zip -r erfnet_training_ce_logitnorm.zip erfnet_training_ce_logitnorm/

### Cross-Entropy Loss + Focal + Logit Norm



In [ ]:
train_erfnet_with_loss(base_dir, data_dir, loss="cef", logit_norm=True)
# !zip -r erfnet_training_ce_focal_logitnorm.zip erfnet_training_ce_focal_logitnorm/

### Cross-Entropy + EIM

In [ ]:
train_erfnet_with_loss(base_dir, data_dir, stop_epoch = 10, loss="ceim", iso_max=True)
# !zip -r erfnet_isomaxplus_training_ceim.zip erfnet_isomaxplus_training_ceim/

In [ ]:
train_erfnet_with_loss(base_dir, data_dir, stop_epoch = 20, loss="ceim", iso_max=True, resume=True)

### Focal + EIM

In [ ]:
train_erfnet_with_loss(base_dir, data_dir, stop_epoch = 10, loss="feim", iso_max=True)
# !zip -r erfnet_isomaxplus_training_feim.zip erfnet_isomaxplus_training_feim/

### Cross-Entropy + Focal + EIM

In [ ]:
train_erfnet_with_loss(base_dir, data_dir, stop_epoch = 10, loss="cefeim", iso_max=True)
# !zip -r erfnet_isomaxplus_training_cefeim.zip erfnet_isomaxplus_training_cefeim/

In [ ]:
train_erfnet_with_loss(base_dir, data_dir, stop_epoch = 20, loss="cefeim", iso_max=True, resume=True)

### Cross-Entropy + Focal + EIM + Logit Norm

In [ ]:
train_erfnet_with_loss(base_dir, data_dir, stop_epoch=10, loss="cefeim", logit_norm=True, iso_max=True)
# !zip -r erfnet_isomaxplus_training_cefeim_logit_norm.zip erfnet_isomaxplus_training_cefeim_logit_norm/

In [ ]:
train_erfnet_with_loss(base_dir, data_dir, stop_epoch=20, loss="cefeim", logit_norm=True, iso_max=True, resume=True)

# Inference with different training losses

In [30]:
load_dir = "../save/"

In [32]:
def run_evaluation_per_loss(losses):
  for loss, (model, training_folder) in losses.items():
    print(f"ERFNet {loss.upper()}")
    plot_folder = training_folder.split("training_")[1]
    run_eval_anomaly(datasets, methods, model, training_folder, plot_folder, load_dir)
    print()
    load_dir_for_iou = f"../save/{training_folder}/"
    model_best = f"model_best.pth"
    run_eval_iou(model, load_dir_for_iou, model_best)
    print()

Evaluate the ERFNet model using various loss functions, such as Cross-Entropy, Focal Loss, Logit Normalization, and their combinations.

In [ ]:
losses = {"Cross-Entropy": ["erfnet", "erfnet_training_ce"],
          "Focal": ["erfnet", "erfnet_training_f"],
          "Cross-Entropy + Focal": ["erfnet", "erfnet_training_cef"],
          "Cross-Entropy + LogitNorm": ["erfnet", "erfnet_training_ce_logit_norm"],
          "Cross-Entropy + Focal + LogitNorm": ["erfnet", "erfnet_training_cef_logit_norm"]}

run_evaluation_per_loss(losses)

ERFNet CROSS-ENTROPY
Dataset SMIYC RA-21
 - MSP        | AUPRC score: 48.367 | FPR@TPR95: 45.054
 - MaxLogit   | AUPRC score: 54.100 | FPR@TPR95: 39.153
 - MaxEntropy | AUPRC score: 52.062 | FPR@TPR95: 44.448
Dataset SMIYC RO-21
 - MSP        | AUPRC score:  9.054 | FPR@TPR95: 11.486
 - MaxLogit   | AUPRC score:  9.520 | FPR@TPR95: 10.512
 - MaxEntropy | AUPRC score:  9.609 | FPR@TPR95: 11.344
Dataset FS L&F
 - MSP        | AUPRC score:  3.003 | FPR@TPR95: 59.098
 - MaxLogit   | AUPRC score:  3.377 | FPR@TPR95: 51.787
 - MaxEntropy | AUPRC score:  4.629 | FPR@TPR95: 58.176
Dataset  FS Static
 - MSP        | AUPRC score:  8.448 | FPR@TPR95: 38.720
 - MaxLogit   | AUPRC score:  7.949 | FPR@TPR95: 48.521
 - MaxEntropy | AUPRC score:  9.671 | FPR@TPR95: 38.744
Dataset Road Anomaly
 - MSP        | AUPRC score: 18.544 | FPR@TPR95: 66.442
 - MaxLogit   | AUPRC score: 20.919 | FPR@TPR95: 58.536
 - MaxEntropy | AUPRC score: 20.075 | FPR@TPR95: 66.105

Loading model: ../save/erfnet_training_ce/e

Evaluate ERFNet with EIM (Enhanced Isotropy Maximation) loss. The variant `erfnet_isomaxplus` is used here.

In [33]:
losses = {"CrossEntropy + EIM": ["erfnet_isomaxplus", "erfnet_isomaxplus_training_ceim"],
          "Focal + EIM": ["erfnet_isomaxplus", "erfnet_isomaxplus_training_feim"],
          "CrossEntropy + Focal + EIM": ["erfnet_isomaxplus", "erfnet_isomaxplus_training_cefeim"],
          "CrossEntropy + Focal + EIM + LogitNorm": ["erfnet_isomaxplus", "erfnet_isomaxplus_training_cefeim_logit_norm"]}

run_evaluation_per_loss(losses)

ERFNet CROSSENTROPY + EIM
Dataset SMIYC RA-21
 - MSP        | AUPRC score: 46.023 | FPR@TPR95: 82.052
 - MaxLogit   | AUPRC score: 37.634 | FPR@TPR95: 89.546
 - MaxEntropy | AUPRC score: 45.868 | FPR@TPR95: 81.474
Dataset SMIYC RO-21
 - MSP        | AUPRC score: 22.832 | FPR@TPR95: 12.967
 - MaxLogit   | AUPRC score: 11.251 | FPR@TPR95: 45.394
 - MaxEntropy | AUPRC score: 24.655 | FPR@TPR95: 12.841
Dataset FS L&F
 - MSP        | AUPRC score:  6.445 | FPR@TPR95: 61.162
 - MaxLogit   | AUPRC score:  6.515 | FPR@TPR95: 47.653
 - MaxEntropy | AUPRC score:  6.113 | FPR@TPR95: 59.712
Dataset  FS Static
 - MSP        | AUPRC score: 12.735 | FPR@TPR95: 31.364
 - MaxLogit   | AUPRC score: 14.345 | FPR@TPR95: 33.941
 - MaxEntropy | AUPRC score: 12.769 | FPR@TPR95: 30.922
Dataset Road Anomaly
 - MSP        | AUPRC score: 20.796 | FPR@TPR95: 69.273
 - MaxLogit   | AUPRC score: 19.166 | FPR@TPR95: 93.703
 - MaxEntropy | AUPRC score: 20.984 | FPR@TPR95: 68.918

Loading model: ../save/erfnet_isomaxpl

# Visualization

In [ ]:
# Example image to color
# Nice images: RoadAnomaly/images/28, RoadAnomaly/images/58
input = '/content/validation_dataset/RoadAnomaly/images/58.jpg'

### Baseline models ###
for method in ["MSP"]: # , "MaxLogit", "MaxEntropy"]:
  print(f"Method: {method}")
  save_image_path = f'/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/plots/images/{method}'

  if torch.cuda.is_available():
    !python -W ignore /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method  {method} --save-colored-anomaly {save_image_path}  | tail -n 2
  else:
    !python -W ignore /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method {method} --save-colored-anomaly {save_image_path} --cpu | tail -n 2

!python /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/plots.py --name_dir="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/baselines/images" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/baseline_visualization.png"

# Ensemble

Perform ensemble inference using ERFNet, ENet, and BiSeNet models trained on the Cityscapes dataset. Their predictions are combined via soft voting (averaging the class probabilities) to compute the per-class IoU and the mean IoU.

In [ ]:
!cd {base_dir} && python -W ignore main_v2.py --datadir {data_dir} --savedir dummy --ensemble

========== ENSEMBLE INFERENCE ===========
Loading ensemble models...
../cityscapes/leftImg8bit/val
Running ensemble inference...
Per-Class IoU:
96.90 Road
80.40 Sidewalk
90.19 Building
49.18 Wall
50.09 Fence
53.24 Pole
54.39 Traffic Light
68.93 Traffic Sign
90.99 Vegetation
61.78 Terrain
93.22 Sky
72.65 Person
48.62 Rider
91.96 Car
72.45 Truck
79.02 Bus
67.49 Train
39.95 Motorcycle
68.90 Bicycle
Ensemble MEAN IoU: 70.02 %
